# Apriori algorithm - Task 1

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import pandas as pd
from scipy.spatial import distance_matrix as dist
from sklearn import tree
from sklearn import datasets,metrics
from scipy.stats import mode as mode
import itertools


In [2]:
%%time
# confidence - conditional probability 
# support - probability
class Apriori:
    def __init__(self,transactions,min_support, min_conf):
        #self.min_lift = min_lift
        self.min_conf = min_conf
        self.min_support = min_support
        self.transactions = [tuple(l) for l in transactions]
        self.supp_for_items = {}
        self.assos_rules = []       
        self.start_dict() # making a dictonary with elements as keys, and support as values
                    
        self.items = [k for k,e in self.supp_for_items.items() if e >= self.min_support] # frequent bought items
        '''
        new_trans = [t for t in transactions if self.contains(t)]
        self.transactions = new_trans
       
        '''
        
        self.rules = {frozenset((x,y)) : self.count_support((x,y)) for index, x in enumerate(self.items) for y in self.items[index:]
                    if x!=y and self.count_support((x,y)) >= min_support}  #  all rules/frequent items sets, at first containing pairs only
        self.L_k = self.rules
        self.K=2
        
    '''
    def contains(self,lista):
        new_trans = []
        for it in self.items:
            if it in lista:
                return True
            else:
                continue
        return False   
     '''

    def make_rules(self):
    #every combination of two set - A,B ze A u B = S i A ^ B = 0
        for k,v in self.rules.items():
            S = k
            len_s = len(S)
            list_s = list(S)
            A = [[] for i in range(len_s-1)]
            for x in S:
                A[0].append(frozenset((x,)))
            for i in range(1,len_s-1):
                for x in S:
                    for e in A[i-1]:
                        if x not in e:
                            new_a = e.union(frozenset((x,)))
                            if new_a not in A[i]:
                                A[i].append(new_a)
            for i in range(len_s-1):
                for a in A[i]:
                    B = S.difference(a)
                    conf = v / self.count_support(a)
                    lift = conf / self.count_support(B)
                    if conf >= self.min_conf:
                        rule = {'sets' : f'{tuple(a)} -> {tuple(B)}','support':v,'confidence':conf, 'lift':lift}
                        self.assos_rules.append(rule)
                        
        print(f'Apriori found {len(self.assos_rules)} rules!')
        
    def start_dict(self):
        for t in self.transactions:
            for it in t:
                if it not in self.supp_for_items:
                    self.supp_for_items[it] = 1
                else:
                    self.supp_for_items[it] += 1
        
    def count_support(self,A):
        a = frozenset(A)
        if a not in self.supp_for_items:
            self.supp_for_items[a] = np.sum([1 for t in self.transactions if a.issubset(t)])
        return self.supp_for_items[a]
    
        
    def Run(self):
        while(self.L_k != {}):
            D_k = {frozenset(x | y) : self.count_support(x | y)
                                           for x in self.L_k for y in self.L_k if(len(x | y)  == self.K+1)} # only these that increase in length
            C_k = {x : D_k[x] for x in D_k.keys() if D_k[x] >= self.min_support} # only these that pass the support threshold
            
            self.L_k = C_k
            for x in self.L_k.keys():
                self.rules[x] = self.L_k[x]
            self.K += 1
            
        self.make_rules()
            



Wall time: 1e+03 µs


# Testing on some small data from wikipedia /wiki/Apriori_algorithm

In [4]:
%%time
sample_transactions = [[1,2,3,4],[1,2,4],[1,2],[2,3,4],[2,3],[3,4],[2,4]]
test = Apriori(sample_transactions,2,0.5)
test.Run()
test.assos_rules
test.rules

Apriori found 17 rules!
Wall time: 5 ms


{frozenset({1, 2}): 3,
 frozenset({1, 4}): 2,
 frozenset({2, 3}): 3,
 frozenset({2, 4}): 4,
 frozenset({3, 4}): 3,
 frozenset({1, 2, 4}): 2,
 frozenset({2, 3, 4}): 2}

# Testing on iris data set with rounded values of attributes

In [40]:
%%time
iris = datasets.load_iris() 
iris.data = [[round(t) for t in l] for l in iris.data] ## rounding up iris attributes
iris_apr = Apriori(iris.data,10,0.65)
iris_apr.Run()
iris_apr.assos_rules

Apriori found 44 rules!
Wall time: 82 ms


[{'sets': '(0.0,) -> (5.0,)',
  'support': 39,
  'confidence': 0.7959183673469388,
  'lift': 0.009706321553011448},
 {'sets': '(5.0,) -> (3.0,)',
  'support': 59,
  'confidence': 0.7195121951219512,
  'lift': 0.006601029313045423},
 {'sets': '(5.0,) -> (2.0,)',
  'support': 59,
  'confidence': 0.7195121951219512,
  'lift': 0.00705404112864658},
 {'sets': '(1.0,) -> (4.0,)',
  'support': 42,
  'confidence': 0.6885245901639344,
  'lift': 0.010758196721311475},
 {'sets': '(4.0,) -> (1.0,)',
  'support': 42,
  'confidence': 0.65625,
  'lift': 0.010758196721311475},
 {'sets': '(1.0,) -> (3.0,)',
  'support': 41,
  'confidence': 0.6721311475409836,
  'lift': 0.00616634080312829},
 {'sets': '(2.0,) -> (3.0,)',
  'support': 71,
  'confidence': 0.696078431372549,
  'lift': 0.006386040654794027},
 {'sets': '(3.0,) -> (2.0,)',
  'support': 71,
  'confidence': 0.6513761467889908,
  'lift': 0.006386040654794027},
 {'sets': '(6.0,) -> (3.0,)',
  'support': 63,
  'confidence': 0.7590361445783133,
  '

# Retail Data - Task 2

In [41]:
%%time
retail_df = pd.read_csv('retail.dat', header=None,names = ['transactions'])
retail_df.head()
retail_df = retail_df.drop([0])
t = retail_df['transactions'].apply(lambda row: list(map(int, row.strip().split(' '))))
# 88161


Wall time: 1.25 s


In [42]:
%%time 
retail_apr = Apriori(list(t[0:88161]),1700,0.8)
retail_apr.Run()
print(retail_apr.assos_rules)
retail_apr.items

Apriori found 8 rules!
[{'sets': '(36,) -> (38,)', 'support': 2790, 'confidence': 0.9502724795640327, 'lift': 6.093052574788617e-05}, {'sets': '(110,) -> (38,)', 'support': 2725, 'confidence': 0.9753042233357194, 'lift': 6.253553624876375e-05}, {'sets': '(170,) -> (38,)', 'support': 3031, 'confidence': 0.9780574378831881, 'lift': 6.271206962574943e-05}, {'sets': '(36, 39) -> (38,)', 'support': 1945, 'confidence': 0.9548355424644085, 'lift': 6.122310480023138e-05}, {'sets': '(110, 39) -> (38,)', 'support': 1740, 'confidence': 0.9891984081864695, 'lift': 6.342641755491598e-05}, {'sets': '(170, 39) -> (38,)', 'support': 2019, 'confidence': 0.9805730937348227, 'lift': 6.287337097555929e-05}, {'sets': '(48, 41) -> (39,)', 'support': 7366, 'confidence': 0.8168108227988468, 'lift': 1.6118615151432594e-05}, {'sets': '(48, 41, 38) -> (39,)', 'support': 1991, 'confidence': 0.8386689132266217, 'lift': 1.6549953887057162e-05}]
Wall time: 23.7 s


[32,
 36,
 38,
 39,
 41,
 48,
 65,
 89,
 101,
 110,
 147,
 170,
 225,
 237,
 270,
 271,
 310,
 413,
 438,
 475,
 1327,
 2238]

## Kosarak Data - Task 3

In [43]:
%%time
kosarak_df = pd.read_csv('kosarak.dat', header=None)
kosarak_df.head()

k = kosarak_df[0].apply(lambda row: list(map(int, row.strip().split(' '))))
data = list(k[:99001])


Wall time: 7.62 s


In [44]:
%%time
kosarak_apr = Apriori(data,3000,0.9)
kosarak_apr.Run()
kosarak_apr.assos_rules

Apriori found 15 rules!
Wall time: 34.8 s


[{'sets': '(148,) -> (6,)',
  'support': 6430,
  'confidence': 0.9312092686459088,
  'lift': 1.5497424920881187e-05},
 {'sets': '(1, 11) -> (6,)',
  'support': 8585,
  'confidence': 0.9416474717560601,
  'lift': 1.567114019032186e-05},
 {'sets': '(1, 218) -> (6,)',
  'support': 3068,
  'confidence': 0.9448721897135818,
  'lift': 1.5724806778617722e-05},
 {'sets': '(11, 7) -> (6,)',
  'support': 5600,
  'confidence': 0.9795347210075215,
  'lift': 1.6301669568092155e-05},
 {'sets': '(27, 7) -> (6,)',
  'support': 3732,
  'confidence': 0.9325337331334332,
  'lift': 1.5519467000622975e-05},
 {'sets': '(27, 11) -> (6,)',
  'support': 4501,
  'confidence': 0.9744533448798441,
  'lift': 1.6217103995470713e-05},
 {'sets': '(64, 11) -> (6,)',
  'support': 3037,
  'confidence': 0.9721510883482715,
  'lift': 1.617878924824044e-05},
 {'sets': '(11, 148) -> (6,)',
  'support': 5476,
  'confidence': 0.9918493026625611,
  'lift': 1.6506612013422997e-05},
 {'sets': '(218, 11) -> (6,)',
  'support': 59